In [1]:
!pip install gdown==4.6.3
!pip install transformers[torch] datasets evaluate

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [2]:
!gdown 1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF

Downloading...
From: https://drive.google.com/uc?id=1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF
To: /content/IMDB_5.csv
100% 323k/323k [00:00<00:00, 91.7MB/s]


In [93]:
import pandas as pd

df = pd.read_csv('/content/IMDB_5.csv')
df.head()

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",negative
1,Why do movie makers always go against the auth...,negative
2,No puedo creer que aquellos alabando esta pelí...,negative
3,Esta película realmente usó sus ubicaciones bi...,positive
4,"De manera extraña, esta película nunca llegó a...",positive


In [94]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [95]:
df['sentiment'] = encoder.fit_transform(df['sentiment'])
df

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",0
1,Why do movie makers always go against the auth...,0
2,No puedo creer que aquellos alabando esta pelí...,0
3,Esta película realmente usó sus ubicaciones bi...,1
4,"De manera extraña, esta película nunca llegó a...",1
...,...,...
245,Obabakoak es un montón de cuentos con un único...,0
246,Todo el mundo civilizado por ahora sabe que es...,1
247,"Soy un enorme fan de las películas de guerra, ...",0
248,"De acuerdo, aquí está el trato: si te ofenden ...",1


In [96]:
df.columns = ['review_es', 'label']

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [97]:
df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)

In [98]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={"train":"train.csv", "test":"test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [99]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_es', 'label'],
        num_rows: 200
    })
    test: Dataset({
        features: ['review_es', 'label'],
        num_rows: 50
    })
})

In [100]:
def tokenize_function(examples):
  return tokenizer(examples['review_es'], padding='max_length', truncation=True, max_length=512)

In [101]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [103]:
from transformers import AutoModelForSequenceClassification

# mozda da se dodae num labels tuka
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [104]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     learning_rate=5e-5,
#     evaluation_strategy="epoch",
#     metric_for_best_model="f1"
# )

# training_args = TrainingArguments("sentiment")
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    metric_for_best_model="f1",
)

In [105]:
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [106]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.896886
2,No log,0.453252
3,No log,0.376053


TrainOutput(global_step=75, training_loss=0.4186103057861328, metrics={'train_runtime': 62.6715, 'train_samples_per_second': 9.574, 'train_steps_per_second': 1.197, 'total_flos': 157866633216000.0, 'train_loss': 0.4186103057861328, 'epoch': 3.0})

In [107]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(50, 2) (50,)


In [108]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [109]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.86, 'f1': 0.8727272727272728}